# How often is X mentioned on Twitter?

#### Load Python tools

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import altair as alt
import altair_stiles as altstiles

In [3]:
alt.themes.register("stiles", altstiles.theme)
alt.themes.enable("stiles")

ThemeRegistry.enable('stiles')

In [4]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000

---

## Get data

#### Counts pulled with the Twarc library. [Check it out](https://twarc-project.readthedocs.io/en/latest/twarc2_en_us/). 

In [5]:
!twarc2 counts --granularity day --csv --archive 'elonmusk OR Elon Musk' data/raw/@elonmusk_mention_counts.csv

100%|███████| Processed 16 years/16 years [03:55<00:00, 269077241 tweets total ]


---

## Read data

#### @elonmusk or 'Elon Musk'

In [6]:
mentioned = "elonmusk"

src = pd.read_csv(
    "data/raw/@elonmusk_mention_counts.csv", parse_dates=["start", "end"]
).sort_values("start", ascending=False)

#### Process dates

In [7]:
src["year"] = pd.to_datetime(src["start"]).dt.strftime("%Y")
src["month_year"] = pd.to_datetime(src["start"]).dt.strftime("%Y-%m")
src["date"] = pd.to_datetime(src["start"]).dt.strftime("%Y-%m-%d")

In [8]:
src[src["date"] == src["date"].max()]

,start,end,day_count,year,month_year,date
30,2022-11-11 00:00:00+00:00,2022-11-11 15:51:00+00:00,900694,2022,2022-11,2022-11-11


In [9]:
src_grouped = (
    src.groupby(["date", "year", "month_year"]).agg({"day_count": sum}).reset_index()
)

#### Slim down and re-order the dataframe

In [10]:
df = (
    src_grouped[["year", "month_year", "date", "day_count"]]
    .sort_values("date", ascending=False)
    .copy()
)

In [11]:
df.head()

,year,month_year,date,day_count
6079,2022,2022-11,2022-11-11,900694
6078,2022,2022-11,2022-11-10,1281179
6077,2022,2022-11,2022-11-09,1012679
6076,2022,2022-11,2022-11-08,1669296
6075,2022,2022-11,2022-11-07,3323372


#### When was the first mention? 

In [12]:
first = df[df["day_count"] > 0]["date"].tail(1).iloc[0]
first

'2007-03-21'

#### How many total mentions? 

In [13]:
df.day_count.sum()

269077241

#### Average mentions? 

In [14]:
df.day_count.mean()

44256.125164473684

#### Make a new dataframe starting from first mention

In [15]:
df_complete = df[df["date"] >= first]

#### Which day was mentioned the most? 

In [16]:
df_complete[df_complete["day_count"] == df_complete["day_count"].max()]

,year,month_year,date,day_count
5880,2022,2022-04,2022-04-26,5032361


In [17]:
df_complete.sort_values("day_count", ascending=False).head(10)

,year,month_year,date,day_count
5880,2022,2022-04,2022-04-26,5032361
6065,2022,2022-10,2022-10-28,4426660
5879,2022,2022-04,2022-04-25,3779527
5882,2022,2022-04,2022-04-28,3705373
6075,2022,2022-11,2022-11-07,3323372
5881,2022,2022-04,2022-04-27,3049417
6066,2022,2022-10,2022-10-29,2551718
6070,2022,2022-11,2022-11-02,2489865
6073,2022,2022-11,2022-11-05,2150186
6069,2022,2022-11,2022-11-01,2132144


#### Chart it

In [43]:
alt.Chart(
    df_complete.head(365 * 2), padding={"left": 10, "top": 10}
).mark_area().encode(
    x=alt.X("date:T", axis=alt.Axis(format="%b. %Y", tickCount=5), title=""),
    y=alt.Y("day_count:Q", title="", axis=alt.Axis(tickCount=5, format="0,M")),
).properties(
    width=650,
    title="Elon Musk mentions in the last two years:",
)

alt.Chart(...)

---

## Aggregate 

#### Groupby month/year

In [19]:
months = (
    df_complete.groupby(["month_year"])
    .agg({"day_count": sum})
    .sort_values("month_year", ascending=False)
    .reset_index()
)

#### Chart it

In [20]:
alt.Chart(months.head(120)).mark_area().encode(
    x=alt.X("month_year:T", axis=alt.Axis(format="%b. %Y", tickCount=5), title=""),
    y=alt.Y("day_count:Q", title=" ", axis=alt.Axis(tickCount=5, format="0,M")),
).properties(width=900, title="Monthly mentions of Elon Musk on Twitter")

/Users/mstiles/.local/share/virtualenvs/notebooks-pUaZ2fM9/lib/python3.9/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

## Exports

In [21]:
months[months["month_year"] > "2014-12"].to_csv(
    f"data/processed/twitter_mentions_{mentioned}_months.csv", index=False
)
df_complete[df_complete["date"] > "2014-12-31"].to_csv(
    f"data/processed/twitter_mentions_{mentioned}_days.csv", index=False
)

In [22]:
months

,month_year,day_count
0,2022-11,20442766
1,2022-10,23951103
2,2022-09,4383841
3,2022-08,5188186
4,2022-07,7393664
5,2022-06,7877539
6,2022-05,19891145
7,2022-04,32622518
8,2022-03,7697786
9,2022-02,5927233


In [23]:
months[months["month_year"] > "2021-01"].day_count.mean()

9484887.727272727